In [4]:
cd ..

/home/user/Documents/workspace/projects/accident-prediction-montreal


In [5]:
from accidents_montreal import get_accident_df
from utils import init_spark
from road_network import get_road_df
from weather import add_weather_columns, extract_year_month_day
from pyspark.sql.functions import row_number, col, rank, avg, split, to_date, min

In [6]:
spark = init_spark()

# accidents

In [7]:
acc_df = get_accident_df(spark)

Skip fetching montreal accidents dataset: already downloaded
Skip extraction of accidents montreal dataframe: already done, reading from file


In [8]:
#acc_df.columns

# dates

In [11]:
#create date_df
dates_df = extract_year_month_day(acc_df.select('DT_ACCDN')
                                  .withColumn('date', to_date(col('DT_ACCDN'), format='yyyy/MM/dd'))).persist()

#get dates lower and upper bound
rows = dates_df.select('year').summary("min", "max").collect()
min_ = rows[0]['year']
max_ = rows[1]['year']
print(min_, max_)

2012 2017


In [12]:
"""# function get_random_date

start = "01/01/" + str(min_)
end = "01/01/" + str(max_)
print(start, end)
start_stamp = datetime.datetime.strptime(start, "%d/%m/%Y").timestamp()
end_stamp = datetime.datetime.strptime(end, "%d/%m/%Y").timestamp()
random.seed()
random_stamp = datetime.datetime.fromtimestamp(random.randint(start_stamp, end_stamp))
date = random_stamp.strftime("%d/%m/%Y")
hour = random.randint(0,12)
print(date, hour)"""

'# function get_random_date\nimport time\nimport datetime\nimport random\nstart = "01/01/" + str(min_)\nend = "01/01/" + str(max_)\nprint(start, end)\nstart_stamp = datetime.datetime.strptime(start, "%d/%m/%Y").timestamp()\nend_stamp = datetime.datetime.strptime(end, "%d/%m/%Y").timestamp()\nrandom.seed()\nrandom_stamp = datetime.datetime.fromtimestamp(random.randint(start_stamp, end_stamp))\ndate = random_stamp.strftime("%d/%m/%Y")\nhour = random.randint(0,12)\nprint(date, hour)'

In [41]:
# generate all dates from start to end
import time
import datetime

start = "01/01/" + str(min_)
end = "31/12/" + str(max_)
date = datetime.datetime.strptime(start, "%d/%m/%Y")

dates = list()
while(date != datetime.datetime.strptime(end, "%d/%m/%Y")): 
    date += datetime.timedelta(days=1)
    for i in range(24):
        dates.append((date.strftime("%Y-%m-%d"), i))

In [42]:
#sc = spark.sparkContext

# take dataframe with 50 dates

In [43]:
from pyspark.sql.functions import udf, col, year
print("nb_dates: ",len(dates))
#spark.createDataFrame(dates, ['date', 'hour']).limit(10).show()
dates_df = spark.createDataFrame(dates, ['date', 'hour'])
print("nb_row in df: ", dates_df.count())
dates_df = dates_df.withColumn('year', year(col('date'))).filter(col('year') == 2017).drop('year')
#print("nb of rows selected: ", dates_df.count())
dates_df.show()
print(dates_df.dtypes)
print("nb selected: ", dates_df.count())

nb_dates:  52584
nb_row in df:  52584
+----------+----+
|      date|hour|
+----------+----+
|2017-01-01|   0|
|2017-01-01|   1|
|2017-01-01|   2|
|2017-01-01|   3|
|2017-01-01|   4|
|2017-01-01|   5|
|2017-01-01|   6|
|2017-01-01|   7|
|2017-01-01|   8|
|2017-01-01|   9|
|2017-01-01|  10|
|2017-01-01|  11|
|2017-01-01|  12|
|2017-01-01|  13|
|2017-01-01|  14|
|2017-01-01|  15|
|2017-01-01|  16|
|2017-01-01|  17|
|2017-01-01|  18|
|2017-01-01|  19|
+----------+----+
only showing top 20 rows

[('date', 'string'), ('hour', 'bigint')]
nb selected:  8760


# location
location (route) => coordinates of a road 

loc_lat, loc_long

In [21]:
from pyspark.sql.functions import rand, monotonically_increasing_id
from preprocess import generate_dates_df

#dates_df = generate_dates_df("01/01/2012", "01/01/2017", spark)
road_df = (get_road_df(spark)
               .select(['center_long', 'center_lat', 'street_id'])
               .withColumnRenamed('center_lat', 'loc_lat')
               .withColumnRenamed('center_long', 'loc_long')
               .orderBy(rand()))
#sc = spark.sparkContext
"""date_rdd = sc.parallelize(dates_df.rdd.take(20)).persist()
road_rdd = sc.parallelize(road_df.rdd.take(20)).persist()
"""

Skip fetching road network: already downloaded
Skip extraction of road network dataframe: already done, reading from file


'date_rdd = sc.parallelize(dates_df.rdd.take(20)).persist()\nroad_rdd = sc.parallelize(road_df.rdd.take(20)).persist()\n'

In [ ]:
#get_road_df(spark).columns

In [25]:
#road_rdd
print("nb of roads", road_df.count())
road_df = road_df.limit(10)
print("nb of roads selected", road_df.count())

nb of roads 539293
nb of roads selected 10


# generation of negatives

nb_roads(10) * nb_times (50)

In [26]:
negatives = dates_df.crossJoin(road_df)
print('nb negatives created : ', negatives.count())

nb negatives created :  500


In [29]:
negatives.sample(0.5).show()

+----------+----+-----------+----------+----------+
|      date|hour|   loc_long|   loc_lat| street_id|
+----------+----+-----------+----------+----------+
|2012-01-02|   0|-73.5526897|45.4736431| 625435875|
|2012-01-02|   0|-73.7609272|45.6791578| 355889550|
|2012-01-02|   0|-73.4866435|45.7275524| 179742563|
|2012-01-02|   0|-73.5164402| 45.705881|1765458084|
|2012-01-02|   0|-73.6797578|45.5570462|1588959189|
|2012-01-02|   0|-73.8508893|45.5077563|1411920530|
|2012-01-02|   0|-73.4922642|45.4196451|  56163489|
|2012-01-02|   0|-73.5058444|45.5847476|1149800350|
|2012-01-02|   1|-73.6797578|45.5570462|1588959189|
|2012-01-02|   1|-73.8508893|45.5077563|1411920530|
|2012-01-02|   1|-73.4922642|45.4196451|  56163489|
|2012-01-02|   1|-73.4771585|45.4493216|1678012933|
|2012-01-02|   2|-73.6797578|45.5570462|1588959189|
|2012-01-02|   2|-73.4922642|45.4196451|  56163489|
|2012-01-02|   3|-73.5526897|45.4736431| 625435875|
|2012-01-02|   3|-73.7609272|45.6791578| 355889550|
|2012-01-02|

In [30]:
#negatives_df = negatives.map(lambda row: row[0] + row[1]).toDF(['date','hour','loc_long','loc_lat', 'street_id']).persist()
negatives = negatives.withColumn('accident_id', monotonically_increasing_id())
negatives.show()

+----------+----+-----------+----------+----------+-----------+
|      date|hour|   loc_long|   loc_lat| street_id|accident_id|
+----------+----+-----------+----------+----------+-----------+
|2012-01-02|   0|-73.5526897|45.4736431| 625435875|          0|
|2012-01-02|   0|-73.7609272|45.6791578| 355889550|          1|
|2012-01-02|   0|-73.4866435|45.7275524| 179742563|          2|
|2012-01-02|   0|-73.7610305|45.4481398| 764139817|          3|
|2012-01-02|   0|-73.5164402| 45.705881|1765458084|          4|
|2012-01-02|   0|-73.6797578|45.5570462|1588959189|          5|
|2012-01-02|   0|-73.8508893|45.5077563|1411920530|          6|
|2012-01-02|   0|-73.4922642|45.4196451|  56163489|          7|
|2012-01-02|   0|-73.5058444|45.5847476|1149800350|          8|
|2012-01-02|   0|-73.4771585|45.4493216|1678012933|          9|
|2012-01-02|   1|-73.5526897|45.4736431| 625435875|         10|
|2012-01-02|   1|-73.7609272|45.6791578| 355889550|         11|
|2012-01-02|   1|-73.4866435|45.7275524|

In [20]:
from pyspark.sql.functions import udf, col, year, month, dayofmonth
from weather import extract_year_month_day
negatives.withColumn('year', year(col('date'))).show()

+----------+----+-----------+----------+----------+-----------+----+
|      date|hour|   loc_long|   loc_lat| street_id|accident_id|year|
+----------+----+-----------+----------+----------+-----------+----+
|2012-01-02|   0|-73.6056889|45.7435093|1869395346|          0|2012|
|2012-01-02|   1|-73.6056889|45.7435093|1869395346|          1|2012|
|2012-01-02|   2|-73.6056889|45.7435093|1869395346|          2|2012|
|2012-01-02|   3|-73.6056889|45.7435093|1869395346|          3|2012|
|2012-01-02|   4|-73.6056889|45.7435093|1869395346|          4|2012|
|2012-01-02|   5|-73.6056889|45.7435093|1869395346|          5|2012|
|2012-01-02|   6|-73.6056889|45.7435093|1869395346|          6|2012|
|2012-01-02|   7|-73.6056889|45.7435093|1869395346|          7|2012|
|2012-01-02|   8|-73.6056889|45.7435093|1869395346|          8|2012|
|2012-01-02|   9|-73.6056889|45.7435093|1869395346|          9|2012|
|2012-01-02|  10|-73.6056889|45.7435093|1869395346|         10|2012|
|2012-01-02|  11|-73.6056889|45.74

In [ ]:
plus_weather = add_weather_columns(spark, negatives)
plus_weather.count()

In [ ]:
from road_network import get_road_features_df
road_features_df = get_road_features_df(spark, road_df=get_road_df(spark))
plus_weather.drop('loc_long').drop('loc_lat').drop('Wind_Chill').join(road_features_df, 'street_id').show()

# weather
weather => year/month/day date + hour
just query the API